In [19]:
import json
from urllib.request import urlopen
import pandas as pd
from pulp import *
import re

In [20]:
def get_url(key, Id=None):
    base_url = 'https://fantasy.premierleague.com/drf'
    api = {
        'All': '/bootstrap-static',
        'Player': '/element-summary/{}',
        'Team':'/my-team/48049/{}',
        '1':'/bootstrap',
        '2':'/bootstrap-static',
        '3':'/bootstrap-dynamic',
        '4':'/events',
        'player_stats':'/elements',
        'player_positions':'/element-types',
        '7':'/fixtures',
        '8':'/teams',
        '9':'/region',
        '10':'/transfers',
        '11':'/entry/{}',
        '12':'/entries',
        '13':'/my-team/{}',
        '14':'/leagues-entered/{}',
        '15':'/leagues-classic/{}',
        '16':'/leagues-classic-standings/{}',
        '17':'/leagues-h2h/{}',
        '18':'/leagues-h2h-standings/{}'
    }
    
    if Id is None:
        url = ''.join([base_url, api[key]])
    else:
        api_extension = api[key]
        api_extension = api_extension.format(Id)
        url = ''.join([base_url, api_extension])
    return url



In [21]:
def pull_data(key, Id=None):
    json_data = json.load(urlopen(get_url(key, Id)))
    df = pd.read_json(json.dumps(json_data))
    return df

In [22]:
def get_player_data(active=False):
    player_stats = pull_data('player_stats')
    player_positions = pull_data('player_positions')[['id', 'singular_name_short']]
    player_table = player_stats.merge(player_positions, left_on='element_type', right_on='id')
    player_table['now_cost'] = player_table['now_cost']/10
    if active:
        player_table = player_table.loc[player_table['chance_of_playing_next_round'] == 100]
        player_table = player_table.reset_index()
    return player_table

In [23]:
active_players = get_player_data(True)

In [24]:
active_players.head()

,index,assists,bonus,bps,chance_of_playing_next_round,chance_of_playing_this_round,clean_sheets,code,cost_change_event,cost_change_event_fall,...,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,web_name,yellow_cards,id_y,singular_name_short
0,0,0,3,130,100.0,100.0,1,11334,0,0,...,78326,60,112108,202,0.0,4.9,Cech,0,1,GKP
1,3,0,2,213,100.0,75.0,4,40349,0,0,...,106712,27,137485,1184,0.6,9.3,Begovic,0,1,GKP
2,6,0,0,0,100.0,100.0,0,50093,0,0,...,29710,118,32093,73,0.0,0.0,Button,0,1,GKP
3,8,0,0,0,100.0,100.0,0,21205,0,0,...,1980,4,5192,11,0.0,0.0,Heaton,0,1,GKP
4,22,0,0,0,100.0,100.0,0,10318,0,0,...,38976,183,76539,213,0.0,0.0,Stekelenburg,0,1,GKP


In [25]:
player_pool = active_players[['web_name', 'singular_name_short', 'now_cost', 'total_points']]
player_pool.columns = ['web_name', 'position', 'cost', 'total_points']
player_pool.head()

,web_name,position,cost,total_points
0,Cech,GKP,4.9,24
1,Begovic,GKP,4.5,42
2,Button,GKP,4.0,0
3,Heaton,GKP,4.8,0
4,Stekelenburg,GKP,4.0,0


In [26]:
costs = {}
points = {}

In [27]:
# Pivot on positions
for pos in player_pool.position.unique():
    pos_players = player_pool.loc[player_pool.position == pos]
    cost = list(pos_players[['web_name', 'cost']].set_index('web_name').to_dict().values())[0]
    point = list(pos_players[['web_name', 'total_points']].set_index('web_name').to_dict().values())[0]
    costs[pos] = cost
    points[pos] = point

In [28]:
#Set number of available players
formation = {
    'GKP': 2,
    'DEF': 5,
    'MID': 5,
    'FWD': 3
}

#Set Budget

BUDGET = 100

In [29]:
#Binary dict defining inclusion of player or not
_vars = {pos: LpVariable.dict(pos, players, cat="Binary") for pos, players in points.items()}

In [30]:
#Starting the problem

prob = LpProblem('Naive_total_score_max', LpMaximize)
rewards = []
Costs = []
position_constraints = []

#Reward

for pos, players in _vars.items():
    Costs += lpSum([costs[pos][player] * _vars[pos][player] for player in players])
    rewards += lpSum([points[pos][player] * _vars[pos][player] for player in players])
    prob += lpSum([_vars[pos][player] for player in players]) <= formation[pos]
    
prob += lpSum(rewards)
prob += lpSum(Costs) <= BUDGET

In [31]:
prob.solve()

1

In [32]:
def summary(prob):
    div = '----------------------------\n'
    print("Variables:\n")
    score = str(prob.objective)
    constraints = [str(const) for const in prob.constraints.values()]
    for v in prob.variables():
        score = score.replace(v.name, str(v.varValue))
        constraints = [const.replace(v.name, str(v.varValue)) for const in constraints]
        if v.varValue != 0:
            print(v.name, "=", v.varValue)
    print(div)
    print("Constraints:")
    for constraint in constraints:
        constraint_pretty = " + ".join(re.findall("[0-9\.]*\*1.0", constraint))
        if constraint_pretty != "":
            print("{} = {}".format(constraint_pretty, eval(constraint_pretty)))
    print(div)
    print("Score:")
    score_pretty = " + ".join(re.findall("[0-9\.]+\*1.0", score))
    print("{} = {}".format(score_pretty, eval(score)))


In [33]:
summary(prob)

Variables:

DEF_Doherty = 1.0
DEF_Duffy = 1.0
DEF_Keane = 1.0
DEF_Laporte = 1.0
DEF_Rüdiger = 1.0
FWD_Agüero = 1.0
FWD_Jiménez = 1.0
FWD_Murray = 1.0
GKP_Dubravka = 1.0
GKP_McCarthy = 1.0
MID_Fraser = 1.0
MID_Martial = 1.0
MID_Richarlison = 1.0
MID_Sigurdsson = 1.0
MID_Sterling = 1.0
----------------------------

Constraints:
4.9*1.0 + 4.7*1.0 + 5.0*1.0 + 6.0*1.0 + 6.0*1.0 + 11.5*1.0 + 5.9*1.0 + 6.7*1.0 + 5.0*1.0 + 4.5*1.0 + 6.2*1.0 + 7.6*1.0 + 7.0*1.0 + 7.4*1.0 + 11.3*1.0 = 99.7
----------------------------

Score:
54*1.0 + 48*1.0 + 57*1.0 + 64*1.0 + 58*1.0 + 82*1.0 + 56*1.0 + 59*1.0 + 48*1.0 + 46*1.0 + 71*1.0 + 61*1.0 + 62*1.0 + 72*1.0 + 95*1.0 = 933.0
